CS669 Representation Learning hw2 code [here](~/Workspace/Class/CS669-RL/hw2/nbs/pg-hw2.ipynb)


## Load libraries

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
InteractiveShell.ast_node_interactivity = 'last_expr'

In [ ]:
import os,sys
import re

sys.dont_write_bytecode = True

In [ ]:
import pandas as pd
# import geopandas as gpd
import joblib

import numpy as np
import matplotlib.pyplot as plt
import cv2

from pprint import pprint
from pathlib import Path

import pdb

In [ ]:
import holoviews as hv
import xarray as xr

from holoviews import opts
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize
from holoviews.streams import Stream, param
from holoviews import streams
import geoviews as gv
import geoviews.feature as gf
from geoviews import tile_sources as gvts


# import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cf

hv.notebook_extension('bokeh')
hv.Dimension.type_formatters[np.datetime64] = '%Y-%m-%d'

# Dashboards
import param as pm, panel as pn
pn.extension()

In [ ]:
# Geoviews visualization default options
H,W, = 250,250
opts.defaults(
    opts.RGB(height=H, width=W, tools=['hover'], active_tools=['wheel_zoom']),
    opts.Image(height=H, width=W, tools=['hover'], active_tools=['wheel_zoom'], framewise=True),#axiswise=True ),
    opts.Points( tools=['hover'], active_tools=['wheel_zoom']),
    opts.Curve( tools=['hover'], active_tools=['wheel_zoom'], padding=0.1),

)

In [ ]:
this_nb_path = Path(os.getcwd())
ROOT = this_nb_path.parent
SCRIPTS = ROOT/'codes'
paths2add = [this_nb_path, SCRIPTS]

print("Project root: ", str(ROOT))
print("this nb path: ", str(this_nb_path))
print('Scripts folder: ', str(SCRIPTS))

for p in paths2add:
    if str(p) not in sys.path:
        sys.path.insert(0, str(p))
        print(str(p), "added to the path\n")
        
# print(sys.path)

In [ ]:
import ipywidgets
from ipywidgets import interact
def f(x):
    return x

interact(f, x=10)

In [ ]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader

from torchvision import transforms, utils
from torch.autograd import Variable

In [ ]:
from datasets import SegDataset

In [ ]:
IMGNET_MEAN = [0.485, 0.456, 0.406]
IMGNET_STD = [0.229, 0.224, 0.225]
TRAIN_DATA_DIR = Path('../data/train/')

- Define train and validation datasets

In [ ]:
train_ds = SegDataset(
    ids_file=TRAIN_DATA_DIR/'train_ids.txt', 
    x_dir=TRAIN_DATA_DIR/'images',
    y_dir=TRAIN_DATA_DIR/'tf_segmentation',
    verbose=True
)

In [ ]:
#sample x,y for testing from now on
train_x, train_y = train_ds[10]


### [SIDE] Caution: Random transforms
When using random transforms, make sure the same transform is applied to both
`x` and `y` pair
- [hack](https://github.com/pytorch/vision/issues/9#issuecomment-383110707)

Below example shows that even the "random" state is set everytime we call the transform object, which means, setting the random seed at the initialization of the transform object doesn't prevent from the randomness to come in when you call the transform on your image. 

Instead if you want to apply the exactly same rotation (say theta) on 1 or more images (eg. in the image segmentation problem, you would like to apply the same rotational transform to your data image (`x`) and label mask image (`y`)), then you need to set a constant, equal seed whenever you apply the transform to your image. 

In [ ]:
%%opts Image [height=400, width=400]
# testing random rotate transformer
np.random.seed(1)
Rotator = transforms.RandomRotation(10)
rot_x = Rotator(train_x)
rot_y = Rotator(train_y)
rot_x_np = np.asarray(rot_x)
rot_y_np = np.asarray(rot_y)
hv.Image(rot_x_np) * hv.Image(rot_y_np).opts(alpha=0.5)

This is the right hack to achieve the same transform


In [ ]:
import random

In [ ]:
%%opts Image [height=400, width=400]
# testing random rotate transformer
tsfms = transforms.Compose([
    transforms.ToPILImage(),
    Rotator])
Rotator = transforms.RandomRotation(10)

seed = random.randint(0,2**32)

random.seed(seed)
rot_x = tsfms(train_x)

random.seed(seed)
rot_y = tsfms(train_y)

rot_x_np = np.asarray(rot_x)
rot_y_np = np.asarray(rot_y)
hv.Image(rot_x_np) * hv.Image(rot_y_np).opts(alpha=0.5)